In [36]:
from bs4 import BeautifulSoup
import requests, re
from consts import *

from user import *
import google.generativeai as genai
from dotenv import load_dotenv
import os, re, json
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage


load_dotenv()


MODEL = "llama-3.1-8b-instant"
llm = ChatGroq(model=MODEL, temperature=0 , api_key= os.environ.get("GROQ_API_KEY"))

In [75]:
def change_date(ini_date):
    l = ini_date.split('-')
    l = l[::-1]

    a = '/'.join(l)
    return a

origin_k = "mumbai"
dest_k = "hyderabad"

origin_mmt = ""
dest_mmt = ""

departure = "2025-07-31"
back = "2025-08-20"

In [76]:
mmt = f"https://www.makemytrip.com/flight/search?tripType=R&itinerary={airports[origin_k.title()]}-{airports[dest_k.title()]}-{change_date(departure)}_{airports[dest_k.title()]}-{airports[origin_k.title()]}-{change_date(back)}&paxType=A-1_C-0_I-0&cabinClass=E"

url = f"https://www.tripozo.com/flight/results?adult=1&child=0&class=1&destination={airports[dest_k.title()]}&fareType=Regular&from={departure}&infant=0&origin={airports[origin_k.title()]}&tripType=oneWay"

In [77]:
mmt

'https://www.makemytrip.com/flight/search?tripType=R&itinerary=BOM-HYD-31/07/2025_HYD-BOM-20/08/2025&paxType=A-1_C-0_I-0&cabinClass=E'

In [58]:
soup = ''
with open("url.html", "r", encoding='utf-8') as f:
    #print(f.read())
    soup = BeautifulSoup(f.read(), 'html.parser')






In [ ]:
flights = soup.find_all("div", class_ = "ant-card-body")
price = soup.find_all("div", class_ = "flight-price-number")
airline = soup.find_all("div", class_ = "airline-name")
fromCity = soup.find_all("div", class_ = "from-airport")
duration = soup.find_all("div", class_ = "f-hvcenter fl-duration")
toCity = soup.find_all("div", class_ = "to-airport")

result = {}

for i in range(len(price)):
    temp = {}
    plane = airline[i].get_text(strip=True) 

    from_airport = fromCity[i]
    codes = from_airport.find_all("div", class_="code")
    airport_code = codes[0].get_text(strip=True)

    # Extract time and city
    time = from_airport.find("div", class_="time").get_text(strip=True)
    city = from_airport.find("div", class_="name").get_text(strip=True)

    match = re.search(r"₹([\d,]+)", str(price[i]))
    p = 0
    if match:
        p = int(match.group(1).replace(",", ""))
        
    flighttime = duration[i].get_text(strip=True)

    to_airport = toCity[i]
    codes = to_airport.find_all("div", class_="code")
    a_airport_code = codes[0].get_text(strip=True)

    # Extract time and city
    a_time = to_airport.find("div", class_="time").get_text(strip=True)
    a_city = to_airport.find("div", class_="name").get_text(strip=True)

    temp = {
        "date" : departure,
        "departure_time" : time,
        "departure" : city,
        "from_airport" : airport_code,
        "price" : p,
        "airline" : plane,
        "duration" : flighttime,
        "to_airport" : a_airport_code,
        "arrival_time" : a_time,
        "arrival" : a_city,
        "arrival_time" : a_time,
        "link" : url
    }

    result[i] = temp




In [33]:
import pandas as pd
import re

In [ ]:
def duration_to_minutes(duration_str):
    h, m = 0, 0
    if 'h' in duration_str:
        h = int(re.search(r'(\d+)h', duration_str).group(1))
    if 'min' in duration_str:
        m_match = re.search(r'(\d+)min', duration_str)
        if m_match:
            m = int(m_match.group(1))
    return h * 60 + m

def final_json():
    df = pd.read_json('data.json')
    df = df.T
    df['duration_min'] = df['duration'].apply(duration_to_minutes)
    df_sorted = df.sort_values(by=['from_airport', 'price', 'duration_min'], ascending=[True, True, True])
    top_flights = df_sorted.groupby('from_airport').head(4).reset_index(drop=True)
    top_flights = top_flights.drop(columns=['duration_min'])
    df = top_flights.T

    df.to_json('okay.json')
    


JSONDecodeError: Extra data: line 1 column 70 (char 69)

In [ ]:


a

'{"0":"2025-08-15","1":"2025-08-15","2":"2025-08-15","3":"2025-08-15"},{"0":"07:40","1":"06:00","2":"22:30","3":"03:00"},{"0":"Mumbai","1":"Mumbai","2":"Mumbai","3":"Mumbai"},{"0":"BOM","1":"BOM","2":"BOM","3":"BOM"},{"0":3833,"1":4970,"2":4970,"3":4970},{"0":"Indigo","1":"Indigo","2":"Indigo","3":"Indigo"},{"0":"2h 0min","1":"Non-Stop","2":"Non-Stop","3":"1h 55min"},{"0":"HDO","1":"DEL","2":"DEL","3":"DEL"},{"0":"09:40","1":"07:55","2":"00:45","3":"05:15"},{"0":"Ghaziabad","1":"Delhi","2":"Delhi","3":"Delhi"},{"0":"https:\\/\\/www.tripozo.com\\/flight\\/results?adult=1&child=0&class=1&destination=DEL&fareType=Regular&from=2025-08-15&infant=0&origin=BOM&tripType=oneWay","1":"https:\\/\\/www.tripozo.com\\/flight\\/results?adult=1&child=0&class=1&destination=DEL&fareType=Regular&from=2025-08-15&infant=0&origin=BOM&tripType=oneWay","2":"https:\\/\\/www.tripozo.com\\/flight\\/results?adult=1&child=0&class=1&destination=DEL&fareType=Regular&from=2025-08-15&infant=0&origin=BOM&tripType=oneWa